In [ ]:
import os
import pickle
from xsg import game, stations
import inspect, re
import collections
from pydoc import locate

filename = './instance/xsg.dat'
line_limit = 50
tab = '- '
GAMES = {}

def limit_str(S):
    x=str(S)
    l=len(x)
    if l > line_limit:
        return (x[:line_limit-3]+'...')
    else:
        return x
    
def save_games_state():
    with open(filename,'wb') as f:
        pickle.dump(GAMES,f)

def load_games_state():
    if os.path.isfile(filename):
        with open(filename,'rb') as f:
            GAMES.clear()
            GAMES.update(pickle.load(f))

def list_items(k,v,l=0):
    if isinstance(v, (str, tuple, int, float)):
        print (tab*l + k + ': ' + limit_str(v))
    elif isinstance(v, list):
        if len(v)>0:
            if isinstance(v[0],(stations.Demand,stations.Station)):
                if k == 'customers':
                    print (tab*l + k + ': ' + limit_str([x.station_name for x in v]))
                elif k == 'suppliers':
                    print (tab*l + k + ': ' + limit_str([x.station_name for x in v]))
            else:
                print (tab*l + k + ': ' + limit_str(v))
        else:
            print (tab*l + k + ': ' + limit_str(v))
    elif isinstance(v, (collections.deque,set)):
        print (tab*l + k + ': ' + limit_str(list(v)))
    elif isinstance(v, (game.Game,stations.Demand,stations.Station)):
        if k == 'game':  # avoid circular ref
            print (tab*l + k + ': ' + limit_str(v.team_name))
        else:
            print (tab*l + k + ':')
            for x in v.__dict__.keys():
                list_items(x,getattr(v,x),l+1)
    elif isinstance(v, dict):
        print (tab*l + k + ':')
        for k1,v1 in v.items():
            list_items(k1,v1,l+1)
    else:
        print ('*'*80+'\ncouldn\'t handle:',k,limit_str(v),str(type(v))+'\n'+'*'*80)
        
def rename_attribute(name,var,parent,name_from,name_to,var_type,parent_type,test=True):
    if name==name_from and isinstance(var,var_type) and isinstance(parent,parent_type):
        if isinstance(parent, (game.Game,stations.Demand,stations.Station)):
            if not test:
                setattr(parent,name_to,var)
                delattr(parent,name_from)
            else:
                print ('***the below is a test only with no impact to the data ***')
            if isinstance(parent, (game.Game)):
                print ('renamed ' + name + ' to ' + name_to + ' in ' + str(parent.team_name))
            elif isinstance(parent, (stations.Demand,stations.Station)):
                print ('renamed ' + name + ' to ' + name_to + ' in ' + str(parent.game.team_name) + '.' + str(parent.station_name))
        elif isinstance(parent, dict):
            if not test:
                parent[name_to]=var
                del parent[name_from]
            else:
                print ('***the below is a test only with no impact to the data ***')
            print ('renamed ' + name + ' to ' + name_to + ' in DICT >>' + str(parent))
        else:
            print ('couldn\t rename variable w/ parent of type: ' + str(type(parent)))

    if isinstance(var, (game.Game,stations.Demand,stations.Station)):
        if name == 'game':  # avoid circular ref
            pass
        else:
            for x in list(var.__dict__.keys()):
                rename_attribute(x,getattr(var,x),var,name_from,name_to,var_type,parent_type,test=test)
    elif isinstance(var, dict):
        for k,v in var.items():
            rename_attribute(k,v,var,name_from,name_to,var_type,parent_type,test=test)
    elif isinstance(var,(list, str, tuple, int, float, collections.deque, set)):
        pass
    else:
        print ('*'*80+'\ncouldn\'t handle:',name,limit_str(var),str(type(var))+'\n'+'*'*80)
        
def add_attribute(obj_name,obj_val,parent_type,attribute_name,attribute_default_value,test=True):
    if isinstance(obj_val,parent_type):
        if isinstance(obj_val, (game.Game,stations.Demand,stations.Station)):
            if not test:
                setattr(obj_val,attribute_name,attribute_default_value)
            else:
                print ('***the below is a test only with no impact to the data ***')
            if isinstance(obj_val, (game.Game)):
                print ('added ' + attribute_name + ' = ' + str(attribute_default_value) + ' in ' + str(obj_val.team_name))
            elif isinstance(obj_val, (stations.Demand,stations.Station)):
                print ('added ' + attribute_name + ' = ' + str(attribute_default_value) + ' in ' + str(obj_val.station_name))
        elif isinstance(obj_val, dict):
            if not test:
                obj_val[attribute_name]=attribute_default_value
            else:
                print ('***the below is a test only with no impact to the data ***')
            print ('added ' + attribute_name + ' = ' + str(attribute_default_value) + ' in DICT >>' + obj_name)
        else:
            print ('couldn\t rename variable w/ parent of type: ' + str(type(obj_val)))

    if isinstance(obj_val, (game.Game,stations.Demand,stations.Station)):
        for x in list(obj_val.__dict__.keys()):
            if x != 'game':  # avoid circular ref
                add_attribute(x,getattr(obj_val,x),parent_type,attribute_name,attribute_default_value,test=test)
    elif isinstance(obj_val, dict):
        for k,v in obj_val.items():
            add_attribute(k,v,parent_type,attribute_name,attribute_default_value,test=test)
    elif isinstance(obj_val,(list, str, tuple, int, float, collections.deque, set)):
        pass
    else:
        print ('*'*80+'\ncouldn\'t handle:',obj_name,limit_str(obj_val),str(type(obj_val))+'\n'+'*'*80) 

In [ ]:
load_games_state()
GAMES.keys()

In [ ]:
# list_items('WSG',GAMES['WSG'])
list_items('GAMES',GAMES)

In [ ]:
rename_attribute('GAMES',GAMES,game.Game,'kpi_fulfillment_rate','kpi_fulfilment_rate',list,stations.Station,test=False)

In [ ]:
import time
# add_attribute('GAMES',GAMES,game.Game,'created',time.time(),test=False)
# add_attribute('GAMES',GAMES,game.Game,'expiry',8760,test=False)
add_attribute('GAMES',GAMES,game.Game,'turn_time',90,test=False)

In [ ]:
save_games_state()